### This file is for quickly cleaning the data without any of the graphs, so that it is easier to quicky create plots in other areas
#### It will save the adjusted DF to a csv for use in other files

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from plotnine import *
%matplotlib inline

#https://www.kaggle.com/datasets/theforcecoder/wind-power-forecasting?resource=download

In [2]:
#Read in the dataframe
df = pd.read_csv("Data/Turbine_Data.csv")
#Rename the Time column from unnamed
df.rename(columns={'Unnamed: 0':'Time'}, inplace = True)
#Convert the time column to a datatime object
df['Time']  = pd.to_datetime(df['Time'])

#Del Columns that provide no info / only 1 value
del(df["WTG"])
del(df["ControlBoxTemperature"])

#Drop all rows that don't have any values
noTime = df.drop('Time', axis=1, inplace=False)
df = pd.DataFrame(df[noTime.notnull().any(axis=1)])

#Since null values are less than 1% for these columns, drop the rows that contain them since they contain no usefull info
df = df.dropna(subset=["ActivePower", "ReactivePower", "WindSpeed"])
#Since these low percent distributions are skewed, replace null with median
nullPer =  df.isnull().sum() / df.shape[0]
lowNullCount = []
for nullP in nullPer:
    if((nullP > 0.0) & (nullP < .05)):
        lowNullCount.append(nullPer[nullPer == nullP].index[0])
for col in lowNullCount:
    df[col].fillna(df[col].median(), inplace=True)
    
# Removing columns that have high correlations and similar distributions
# among the correlated columns, we remove the ones with lowest null values 
to_drop = ['GearboxOilTemperature','Blade2PitchAngle', 'Blade3PitchAngle','GeneratorWinding1Temperature',
           'RotorRPM','NacellePosition']
# creating new dataframe by dropping the above columns
df_new = df.drop(to_drop, axis=1, inplace=False)
df = pd.DataFrame(df_new)

#removing outliers using the interquartile range
cols = list(df.columns)
[cols.remove(x) for x in ["Time"]]
Q3 = df[cols].quantile(0.75)
Q1 = df[cols].quantile(0.25)
iqr=Q3-Q1
df = df[~((df[cols] < (Q1 - 1.5 * iqr)) |(df[cols] > (Q3 + 1.5 * iqr))).any(axis=1)]

# multiple columns had large % of missing values for the same observations
# dropping rows with null values for the column with the least number of nans
df = df.dropna(subset=["BearingShaftTemperature"])

# imputing rest of the data
# interpolation works best with this data set
df.interpolate(method='pad', limit_direction='forward', axis = 0, inplace=True)
# impute remaining with median
for col in df.columns[1:]:
    df[col].fillna(df[col].median(), inplace=True)

# the distribuitions still show outliers on the extreme ends so we remove outliers using percentile
cols = list(df.columns)
[cols.remove(x) for x in ["Time"]]
upper = df[cols].quantile(0.99)
lower = df[cols].quantile(0.01)
df = df[~((df[cols] < lower) |(df[cols] > (upper))).any(axis=1)]

#Expand upon the time columns to make it easier for the time series graphs
df["year"] = df.Time.dt.year
df["month"] = df.Time.dt.month
df["day"] = df.Time.dt.day
df.to_csv("Data/cleanData.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41915 entries, 33362 to 118223
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype              
---  ------                        --------------  -----              
 0   Time                          41915 non-null  datetime64[ns, UTC]
 1   ActivePower                   41915 non-null  float64            
 2   AmbientTemperatue             41915 non-null  float64            
 3   BearingShaftTemperature       41915 non-null  float64            
 4   Blade1PitchAngle              41915 non-null  float64            
 5   GearboxBearingTemperature     41915 non-null  float64            
 6   GeneratorRPM                  41915 non-null  float64            
 7   GeneratorWinding2Temperature  41915 non-null  float64            
 8   HubTemperature                41915 non-null  float64            
 9   MainBoxTemperature            41915 non-null  float64            
 10  ReactivePower                

In [4]:
# import dataframe_image as dfi
# dfi.export(df.describe(), 'summary.png')

In [5]:
# pip install dataframe-image

In [6]:
print(df['Time'].min())
print(df['Time'].max())

2018-08-19 16:20:00+00:00
2020-03-30 23:50:00+00:00
